# CNN

kaggle: https://www.kaggle.com/competitions/ml2021spring-hw3/overview

In [39]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset, Dataset, Subset
from PIL import Image
# # 合并数据集用
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torchvision import models

from tqdm.auto import tqdm

## Dataset, DataLoader, Transforms

In [40]:
train_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    
    transforms.RandomHorizontalFlip(), # 隨機將圖片水平翻轉
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

In [41]:
batch_size = 128

# 构建数据集
train_set = DatasetFolder('./food-11/training/labeled', loader=lambda x: Image.open(x), extensions='jpg', transform=train_tfm)
val_set = DatasetFolder('./food-11/validation', loader=lambda x: Image.open(x), extensions='jpg', transform=test_tfm)
unlabeled_set = DatasetFolder("./food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder('./food-11/testing', loader=lambda x: Image.open(x), extensions='jpg', transform=test_tfm)

# 构建数据加载器
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## Model

In [42]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # input image size: [3, 128, 128]
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(512, 11)

    def forward(self, x):
        return self.model(x)

## Training

使用 semi-supervised learning 提高performance

In [43]:
class PseudoDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


def get_pseudo_label(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    softmax = nn.Softmax(dim=1)
    for i, batch in tqdm(data_loader):
        img, _ = batch
        with torch.no_grad():
            logit = model(img.to(device))
        probs = softmax(logit)  # size: batch_size x 11
        
        idx = []
        labels = []
        for j, x in enumerate(probs):
            if torch.max(x) > threshold:
                idx.append(i * batch + j)
                labels.append(int(torch.argmax(x)))

    model.train()
    dataset = PseudoDataset(Subset(dataset, idx), labels)
    return dataset

In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
epochs = 50
do_semi = False

for epoch in range(epochs):
    if do_semi:
      # ---------for semi-supervised learning--------
      # Not finish
        pseudo_set = get_pseudo_label(train_set, model)
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True)
      # ---------for semi-supervised learning--------


    model.train()
    train_loss = []
    train_acc = []

    for batch in tqdm(train_loader):
        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        # 梯度剪裁
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        train_loss.append(loss.item())
        train_acc.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_acc) / len(train_acc)
    
    print(f"[ Train | {epoch + 1:03d}/{epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")


    model.eval()
    valid_loss = []
    valid_acc = []
    for batch in tqdm(valid_loader):
        imgs, labels = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            valid_loss.append(loss.item())
            valid_acc.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_acc) / len(valid_acc)

    print(f"[ Valid | {epoch + 1:03d}/{epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 001/050 ] loss = 2.38299, acc = 0.16813


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 001/050 ] loss = 2.45667, acc = 0.13125


  0%|          | 0/25 [00:00<?, ?it/s]

...

[ Valid | 049/050 ] loss = 1.24270, acc = 0.59375


  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 050/050 ] loss = 0.99852, acc = 0.66875


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 050/050 ] loss = 1.35193, acc = 0.59688


## Testing

In [47]:
model.eval()

predictions = []

for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    with torch.no_grad():
        logits = model(imgs.to(device))

    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

predictions[:10]

  0%|          | 0/27 [00:00<?, ?it/s]

[2, 10, 4, 9, 7, 3, 4, 10, 0, 2]

In [48]:
with open("predict.csv", "w") as f:
    f.write("Id,Category\n")

    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")

In [51]:
!kaggle competitions submit ml2021spring-hw3 -f ./predict.csv -m 'ResNet18 with no pre-train'

100% 22.1k/22.1k [00:01<00:00, 19.8kB/s]
Successfully submitted to ML2021spring - hw3